In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import pytz
from pytz import common_timezones, all_timezones
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline
from datetime import datetime
import scipy as sp
import statsmodels.api as sm
import statsmodels.formula.api as smf
matplotlib.style.use('fivethirtyeight')
matplotlib.style.use('seaborn-talk')
import os
import collections

In [32]:
preDouble = "\\\\iobsdc01\\SharedDocs\\SEER_MEDICARE\\SEER_MEDICARE_STAGING"
new = os.chdir(preDouble)
print(preDouble)

\\iobsdc01\SharedDocs\SEER_MEDICARE\SEER_MEDICARE_STAGING


In [33]:
import pickle
fileObject = open('control_indicies.pickle', 'rb')
b = pickle.load(fileObject)
fileObject.close()
controlindices = list(b)

In [34]:
import pickle
fileObject = open('treatment_indicies.pickle', 'rb')
b = pickle.load(fileObject)
fileObject.close()
treatmentindices = list(b)

In [35]:
print(len(controlindices), len(treatmentindices))

1031 1031


Want to get the concatenated outsaf files for only these patients.

In [36]:
import glob
#filelist = glob.glob('./outsaf*')
filelist = glob.glob('outsaf*')
goodfilelist = [f for f in filelist if f.endswith('.txt')]
goodfilelist[:15]

['outsaf06.file001.txt',
 'outsaf06.file002.txt',
 'outsaf06.file003.txt',
 'outsaf06.file004.txt',
 'outsaf06.file005.txt',
 'outsaf06.file006.txt',
 'outsaf06.file007.txt',
 'outsaf06.file008.txt',
 'outsaf06.file009.txt',
 'outsaf06.file010.txt',
 'outsaf06.file011.txt',
 'outsaf06.file012.txt',
 'outsaf06.file013.txt',
 'outsaf06.file014.txt',
 'outsaf06.file015.txt']

In [37]:
specsdict = {'patient_id': (0,0+11),
            'casereg': (0,0+2),
             'claim_from_date': (31,31+8),
             'claim_through_date': (39,39+8),
             'fac_type': (59,59+1),
             'freq_cd': (61,61+1),
             'pmt_amt': (64,64+15),              
             'tot_chrg': (79,79+15),
             'tot_line': (174,174+3),
             'seg_line': (177,177+2),
             'e1dgnscd': (198,198+7),
             'opsrvtyp': (216,216+1),
             'center': (218,218+4),
             'hcpcs': (240,240+5),
             'mf1': (245,245+2),
             'mf2': (247,247+2),
             'mf3': (249,249+2),
             'charge': (372,372+15),
             'revncvr': (402,402+15),
            'year': (421,421+4),
            'link_num': (425,425+10),
             'rec_count': (447,447+3),
             'tot_seg': (443,443+2),
             'seg_num': (445,445+2),
              'prvdrpmt': (495,495+15),
            'edgnsd1': (743,743+7),
            'edgnsd2': (750,750+7),
            'edgnsd3': (757,757+7),
            'prcdr_cd1': (785,785+7),
            'prcdr_cd2': (785+7, 785 + 2*7),
            'prcdr_cd3': (785+2*7, 785+3*7)}

names = sorted(specsdict,key=specsdict.__getitem__)

colspecs = [specsdict[n] for n in names]

patient_id, claim from date, linknum, and rec_count
is the key

In [38]:
df = pd.read_fwf(goodfilelist[1], colspecs = colspecs,
                names=names)

df.patient_id = df.patient_id.astype(str)
df.patient_id = df.patient_id.apply(lambda x: x.rjust(10, '0'))

df.set_index('patient_id', inplace=True,drop=False)

Need to just keep the subset of df that corresonds to the controlindices and treatmentindices.

In [39]:
df.shape

(520000, 31)

In [40]:
df.head(10)

,casereg,patient_id,claim_from_date,claim_through_date,fac_type,freq_cd,pmt_amt,tot_chrg,tot_line,seg_line,...,tot_seg,seg_num,rec_count,prvdrpmt,edgnsd1,edgnsd2,edgnsd3,prcdr_cd1,prcdr_cd2,prcdr_cd3
patient_id,,,,,,,,,,,,,,,,,,,,,
0263468060,2,0263468060,8222006,8222006,1,1,798.05,4337.20,7,7,...,1,1,7,798.05,NaN,NaN,NaN,NaN,NaN,NaN
0263468060,2,0263468060,10302006,10302006,7,2,90.03,219.00,4,4,...,1,1,1,90.03,NaN,NaN,NaN,NaN,NaN,NaN
0263468060,2,0263468060,10302006,10302006,7,2,90.03,219.00,4,4,...,1,1,2,90.03,NaN,NaN,NaN,NaN,NaN,NaN
0263468060,2,0263468060,10302006,10302006,7,2,90.03,219.00,4,4,...,1,1,3,90.03,NaN,NaN,NaN,NaN,NaN,NaN
0263468060,2,0263468060,10302006,10302006,7,2,90.03,219.00,4,4,...,1,1,4,90.03,NaN,NaN,NaN,NaN,NaN,NaN
0263468060,2,0263468060,4102006,4102006,1,3,88.07,423.25,6,6,...,1,1,1,88.07,NaN,NaN,NaN,NaN,NaN,NaN
0263468060,2,0263468060,4102006,4102006,1,3,88.07,423.25,6,6,...,1,1,2,88.07,NaN,NaN,NaN,NaN,NaN,NaN
0263468060,2,0263468060,4102006,4102006,1,3,88.07,423.25,6,6,...,1,1,3,88.07,NaN,NaN,NaN,NaN,NaN,NaN
0263468060,2,0263468060,4102006,4102006,1,3,88.07,423.25,6,6,...,1,1,4,88.07,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
df.hcpcs.value_counts()

36415    22884
85025    13831
97110    13436
85610    10116
80053     7887
77418     7605
80061     6668
82565     6632
84520     6186
97140     5448
77413     5123
77417     4775
80048     4593
80051     4530
77414     4526
88305     4396
84443     4304
71020     3949
85027     3747
93005     3701
82947     3643
84460     3626
77336     3581
84450     3531
99213     3398
83036     3393
97530     3095
85730     2695
99214     2665
97035     2658
         ...  
A4321        1
36100        1
83909        1
42826        1
84202        1
69320        1
99374        1
76100        1
75984        1
24505        1
31646        1
84588        1
86886        1
27340        1
74425        1
90376        1
73201        1
31717        1
87281        1
95925        1
42440        1
75860        1
90691        1
L3800        1
L3763        1
67255        1
97761        1
L3650        1
58353        1
38520        1
Name: hcpcs, dtype: int64

In [42]:
goodindices = controlindices + treatmentindices
print(len(goodindices),len(controlindices),len(treatmentindices))

2062 1031 1031


In [43]:
list(set(df.index).intersection(set(goodindices)))

['0292469070',
 '0268487040',
 '0280413070',
 '0299434050',
 '0272492080',
 '0274484060',
 '0267404090',
 '0273472060',
 '0277407050']

In [44]:
df.loc[list(set(df.index).intersection(set(goodindices)))].shape

(204, 31)

In [ ]:
def get_good_outclaims(fin,goodindices):
    """Takes a file name, loads it and returns only
    the parts corresponding to goodindices."""
    df = pd.read_fwf(fin, colspecs = colspecs,
                names=names)
    df.patient_id = df.patient_id.astype(str)
    df.patient_id = df.patient_id.apply(lambda x: x.rjust(10, '0'))

    df.set_index('patient_id', inplace=True,drop=False)
    
    
    
    df = df.loc[list(set(df.index).intersection(set(goodindices)))]
    
    print(df.shape)
    print(list(set(df.index).intersection(set(goodindices))))
    
    return df

In [ ]:
allframeslist = []

for i in range(len(goodfilelist)):
    di = get_good_outclaims(goodfilelist[i],goodindices)
    allframeslist.append(di)
    print(i, di.shape, len(allframeslist))
    del di

Concatenate the above dataframe.

In [ ]:
len(allframeslist)

In [ ]:
a = allframeslist

In [ ]:
dg = pd.concat(a, ignore_index=True)

In [ ]:
dg.shape

In [ ]:
dg.to_pickle('goodindices_costofcare_outsaf.pickle')

In [ ]:
dg.head(15)